In [3]:
%matplotlib inline

import os, json
import sys

# the ~ refers to your home directory in the linux prompt but jupyter does not recognize that symbol, 
# you either provide a full path (something like /home/yoursusename/data//dogscats/sample) 
# or a correct relative path from the location where you launch jupyter

path = os.path.realpath("../../data/dogs-vs-cats-redux/")
os.listdir(path)
print(path)

#sys.exit()


from __future__ import division,print_function

#import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

import utils; reload(utils)
from utils import plots

#print (os.listdir())

from vgg16 import Vgg16
#vgg = Vgg16()
#batches = vgg.get_batches(path+'/train/', batch_size=4,shuffle=False)
#imgs,labels = next(batches)
#vgg.predict(imgs, True)

/home/pmaroun/data/dogs-vs-cats-redux


Using Theano backend.


In [4]:
vgg = Vgg16()
batch_size=64
batches = vgg.get_batches(path+'/train/', batch_size=batch_size)
val_batches = vgg.get_batches(path+'/valid/', batch_size=batch_size)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)


/home/pmaroun/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Epoch 1/1
22500/22500 [==============================] - 11135s - loss: 0.1201 - acc: 0.9675 - val_loss: 0.0751 - val_acc: 0.9808

In [5]:
def predict(base_train_folder, vgg):
    from keras.preprocessing import image
    import pandas as pd
    imags = glob(os.path.join(base_train_folder, 'test','*.jpg'))

    records = []
    for n, path_img in enumerate(imags):
        probs = vgg.model.predict(
            image.img_to_array(image.load_img(path_img, target_size=[224, 224])).reshape(1, 3,
                                                                         224,
                                                                         224))
        base = os.path.basename(path_img)
        number = os.path.splitext(base)[0]
        
        records.append({'id': number, 'label': probs[0][1]})
    
        print ({'id': number, 'label': probs[0][1]})

    print(len(records))
    df = pd.DataFrame.from_records(records)
    df['id'] = pd.to_numeric(df['id'])
    df = df.sort_values('id')
    df.to_csv('submission.csv', index=False)

In [6]:
predict(path, vgg)

{'id': '1', 'label': 0.99999982}
{'id': '10', 'label': 1.0444975e-10}
{'id': '100', 'label': 1.246787e-05}
{'id': '1000', 'label': 1.0}
{'id': '10000', 'label': 0.99999708}
{'id': '10001', 'label': 3.6562642e-07}
{'id': '10002', 'label': 1.1709654e-09}
{'id': '10003', 'label': 0.99990404}
{'id': '10004', 'label': 0.99999994}
{'id': '10005', 'label': 1.2228264e-10}
{'id': '10006', 'label': 2.050523e-07}
{'id': '10007', 'label': 5.6372457e-10}
{'id': '10008', 'label': 6.7600057e-12}
{'id': '10009', 'label': 2.0015066e-13}
{'id': '1001', 'label': 3.0723224e-13}
{'id': '10010', 'label': 0.99999934}
{'id': '10011', 'label': 0.99995571}
{'id': '10012', 'label': 1.9566145e-10}
{'id': '10013', 'label': 4.2311186e-08}
{'id': '10014', 'label': 9.8291209e-10}
{'id': '10015', 'label': 1.0}
{'id': '10016', 'label': 3.6502473e-07}
{'id': '10017', 'label': 1.0123263e-11}
{'id': '10018', 'label': 1.0133329e-07}
{'id': '10019', 'label': 1.879815e-09}
{'id': '1002', 'label': 1.0}
{'id': '10020', 'label'